In [3]:
## model init
from langchain.chat_models import init_chat_model

nano_model = init_chat_model(
    model="gpt-5-nano",
    temperature=0.5,
    
)

mini_model = init_chat_model(
    model="gpt-5-mini",
    temperature=0.5
)



In [46]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

@wrap_model_call
def dynamic_model_selection(request:ModelRequest, handler)->ModelResponse:
    """Choose the model base don the converstion complexity"""
    message_len = len(request.state["messages"])
    print("Messgaes len : ", message_len)
    if message_len > 1 : 
        model = mini_model
    else:
        model = nano_model

    return handler(request.override(model=model))

In [47]:
from langchain.agents import create_agent

agent = create_agent(
    model=mini_model, ## default model
    middleware=[dynamic_model_selection]
)

In [48]:
out = agent.invoke({"messages" : [{"role": "user", "content": "Im a airplane pilot"},
                                  {"role": "user", "content": "Who am I?"}]})

Messgaes len :  2


In [49]:
out["messages"][-1].__dict__

{'content': 'Literally: you’re an airplane pilot — an aviator who flies aircraft.\n\nIf you mean a more specific label, it depends on your license and job:\n- Student pilot (learning)\n- Private pilot (recreational)\n- Commercial pilot (paid to fly)\n- Airline transport pilot (ATP, airline captain/first officer)\n- Flight instructor (CFI)\n- Military pilot\n- Cargo, charter, corporate, or helicopter/aircraft-specific roles\n\nDo you mean “Who am I?” psychologically or in a career sense? If you want I can help identify your exact title (tell me what aircraft you fly, your certificate/ratings, and whether you fly for an airline, charter, cargo, corporate or privately), or I can give career, training, or resume/interview advice. Which would you like?',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 495,
   'prompt_tokens': 18,
   'total_tokens': 513,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 

In [50]:
# define tool
from langchain.tools import tool

@tool
def search(query:str)->str:
    """Search the query Information"""
    return f"The search result si in the air"

@tool
def get_weather(location:str)->str:
    """Get relatime weather of the location"""
    return f"The weather od {location} is windy"

In [51]:
# how to handle tool errors
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

agent = create_agent(
    model="gpt-4o",
    tools=[search, get_weather],
    middleware=[handle_tool_errors]
)

In [53]:
agent.invoke({"messages": "Get the weather in kochi?"})

{'messages': [HumanMessage(content='Get the weather in kochi?', additional_kwargs={}, response_metadata={}, id='832fe2e8-81be-4eeb-b86b-327dcf669ae1'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 70, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-D0jSDh2GNr4NICnKdJCHiXmbOLpLO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019be49d-a904-74d3-8f9a-f7920148d900-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Kochi'}, 'id': 'call_VsOrleO35rMeawzuTpRCAiEt', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 70, 'output_toke